API hh.ru нужно две недели для предоставления токена доступа и вообще она не очень адаптирована для парсинга вакансий (больше под работодателей), так что мы не ищем легких путей)

In [1]:
import fake_useragent
from bs4 import BeautifulSoup
import requests
import time
import json
from tqdm import tqdm

In [42]:
#получаем ссылки на нужные вакансии
def get_links(text):
    #используем useragent, чтобы больше походить на человека
    ua = fake_useragent.UserAgent()
    #ссылка на страницу для парсинга (с номером страницы)
    res = requests.get(
        url = f"https://hh.ru/vacancies/{text}?page=1",
        headers = {"user-agent":ua.random}
    )
    #условие того, что сервер вернул данные  
    if res.status_code != 200:  return
    #передаем содержание страницы (дерево)
    soup = BeautifulSoup(res.content, "html")
    #считаем количество страниц с вакансиями администраторов
    try:
        finded_div = soup.find("div", attrs = {"class":"pager"}) 
        cnt_pages = int(finded_div.find_all("span", recursive = False)[-1].find("a").find("span").text)
    except:
        return
    #идем по всем страницам
    for page in range(cnt_pages):
        #собираем вакансии со страницы
        try:
            res = requests.get(
            url=f"https://hh.ru/vacancies/{text}?page={page}",
            headers={"user-agent":ua.random}
            )
            if res.status_code != 200:
                continue
            else:
            #собираем ссылки на все вакансии с конкретной страницы
                soup = BeautifulSoup(res.content, "html")
                for i in soup.find_all('a', attrs = {"class": "bloko-link", "target": "_blank"}):
                    l = f"{i.attrs['href'].split('?')[0]}" #создаем ссылки на вакансии
                    if 'vacancy' in l: #помимимо нужных ссылок у меня парсились ссылки на фидбэк и еще несколько странных, не хочется их возвращать
                        yield l 
        except Exception as e: print(f'e')
        time.sleep(1)  #пытаемся еще больше быть похожим на человека
#получаем страницу вакансии по ссылке и парсим с нее нужную нам информацию
def get_vacancy(link):
    #используем useragent, чтобы больше походить на человека
    ua = fake_useragent.UserAgent()
    #получаем страницу конвертной вакансии
    data = requests.get(
        url = link,
        headers = {"user-agent":ua.random}
    )
    #получилось ли спарсить информацию со страницы
    if data.status_code != 200: return
    soup = BeautifulSoup(data.content, 'html')
    #получаем названия вакансий
    try:
        name = soup.find(attrs = {"class":"bloko-header-section-1"}).text
    except:
        name = ""
    #получаем зарплату указанную в вакансии
    try:
        #зарплата парситься в неудобном формате, поэтому используем replace, чтобы потом было проще обрабатывать данные
        salary = soup.find(attrs = {"class":"bloko-header-section-2 bloko-header-section-2_lite"}).text.replace('\xa0', '') 
    except:
        salary = ""
    #получаем набор ключевых навыков (на hh.ru есть соответствующее поле)
    try:
        #сначала находим на странице необходимое поле, потом все навыки из него записываем в массив skills
        skills = [skill.text for skill in soup.find(attrs = {"class":"bloko-tag-list"}).find_all(attrs = {"class":"bloko-tag bloko-tag_inline"})]
    except:
        skills = []
    #создаем обьект класса json - вакансия, с интересующей нас информацией: название, з/п, ключевые навыки
    vacancy = {
        "name": name,
        "salary":salary,
        "skills":skills
    }
    return vacancy

#создаем массив с данными, который будем заполнять
data = []
#выводим спаршенные данные в json файл
for link in tqdm(get_links("administrator")):
    data.append(get_vacancy(link))
    #примерно раз в секунду будет парситься новая вакансия: очень долго (около часа),
    # но зато мы теперь очень похожи на человека, скролящего вакансии администраторов
    time.sleep(1)
    with open ('hh_data.json', 'w', encoding = 'utf-8') as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii = False) #не хотим, чтобы русские буквы на что-то заменялись

800it [22:24,  1.71s/it]

e
e


1900it [52:10,  1.65s/it]


Парсер падал только два раза (из-за того, что я отклюался от VPN). При этом количество вакансий, указанных на hh не совпадает с количеством вакансий, который находятся на 40 открытых страницах